In [ ]:
import matplotlib.pyplot as plt
import matplotlib 

import numpy as np
import pickle
import time
import os

from matplotlib_setup import set_size, configure_latex, savefig, generate_tex_figures, thiner_border

configure_latex(style=['science', 'notebook'], global_save_path=os.getcwd() + "/porocilo/images")

In [ ]:
from methods import MultiNSaverLoader

names = ['adi_tridiag', 'adi_sparse', 'iter_diff', 'matrix_diff', 'ssfm']
dcts = {n: {} for n in names}

loader = MultiNSaverLoader()

for k in dcts.keys():
    dcts[k].update({'times': loader.load('times_' + k)})
    dcts[k].update({'norms': loader.load('norms_' + k)})
    dcts[k].update({'max': loader.load('max_' + k)})
    dcts[k].update({'E': loader.load('Es_' + k)})

In [ ]:
def get_metric(m, dcts, names, ax):
    metric = []
    
    for n in names:
        lst = []
        
        for i in dcts[n][m]:
            if m == 'norms':
                lst.append(np.mean(np.diff(i)))
            elif m == 'max':
                lst.append(np.mean(i - i[0]))
            elif m == 'times':
                lst.append(np.abs(i))
            else:
                raise ValueError
        
        metric.append(lst)
        
    for i, n in enumerate(names):
        ax.scatter(range(len(metric[i])), 50000 / np.array(metric[i]), s=10)
        ax.plot(range(len(metric[i])), 50000 / np.array(metric[i]), lw=1)

In [ ]:
Ns = loader.load('norms_at_ssfm')

fig, ax = plt.subplots(1, 1, figsize=set_size(fraction=0.5, ratio='4:3'))
ax = thiner_border(ax)

names_ = names # ['iter_diff', 'matrix_diff']

get_metric('times', dcts, names_, ax)

x = [i for i in range(len(Ns))]

ax.set_xticks(x[::3])
ax.set_xticklabels(Ns[::3])

ax.set_yscale('log')

ax.legend(['ADI tridiag', 'ADI sparse', 'DIFF', 'DIFF sparse', 'SSFM'], fontsize=5)

ax.set_xlabel('$N$')
ax.set_ylabel('iteracije / s')

savefig('times_all')

In [ ]:
from methods import Method

L, T, N = 5, 10, 128

n, a_x, a_y, lam = 0, 1, 0, 0

methods = ['sparse_adi', 'tridiag_adi', 'iter_diff', 'matrix_diff', 'ssfm']

r = []

for m in methods:
    print(m)
    M = Method(N=N, Nt=N, method_name=m)
    M.init_grid(L=L, T=T)
    M.init_pot(n=n, a_x=a_x, a_y=a_y, lam=lam)
    
    if m == 'tridiag_adi':
        check_overflow = 1
    else:
        check_overflow = 1.025
        
    if m == 'tridiag_adi':
        M = M.find_stable_time_step(Nt_min=10000, Nt_max=20000, save_every=None, check_overflow=check_overflow, it=3)
    else:
        M = M.find_stable_time_step(Nt_min=1000, Nt_max=20000, save_every=None, check_overflow=check_overflow, it=12)
    
    r.append(M)

In [ ]:
from utils import psi_norms, max_psi_values

def plot_max_values(ax, psi_lst, N, k=10):
    max_vals = max_psi_values(psi_lst, N)
    ax.plot(range(len(max_vals)), max_vals, lw=1, c='C2')
    ax.set_xlabel(r'koraki $\times$ {}'.format(k))
    ax.set_ylabel(r'$\textrm{max} [\psi^0] - \textrm{max} [\psi^{n}]$')

def plot_norms(ax, psi_lst, hr, minus_one=True, k=10):
    norms = psi_norms(psi_lst, hr)
    ax.plot(range(len(norms)), (1 - norms) if minus_one else norms, lw=1, c='C3')
    ax.set_xlabel(r'koraki $\times$ {}'.format(k))
    ax.set_ylabel(r'$1 - \parallel \psi \parallel^2$')
    
def plot_Es(axs, E, E_k, E_p, norm=False, k=10):
    x = range(len(E))

    axs[0].plot(x, E, lw=1, c='C4')

    axs[1].plot(x, E_p / np.max(np.abs(E_p)) if norm else E_p, label="$V$", lw=1)
    axs[1].plot(x, E_k / np.max(np.abs(E_k)) if norm else E_k, label="$T$", lw=1)
    axs[1].legend()
    
    axs[0].set_xlabel(r'koraki $\times$ {}'.format(k))
    axs[0].set_ylabel('skupna energija')
    
    axs[1].set_xlabel(r'koraki $\times$ {}'.format(k))
    axs[1].set_ylabel('energija')

In [ ]:
from utils import calculate_E
import copy 

for m in r:
    print(m.method_name)
    
    M = Method(N=N, Nt=m.Nt, method_name=m.method_name)
    M.init_grid(L=L, T=T)
    M.init_pot(n=n, a_x=a_x, a_y=a_y, lam=lam)

    M.start_method(check_overflow=False, save_every=100 if m.method_name != 'ssfm' else 10)
    
    fig, axs = plt.subplots(1, 2, figsize=set_size(subplots=(1, 2), fraction=1, ratio='4:3'))
    axs = [thiner_border(i) for i in axs]
    
    plot_norms(axs[0], M.res, M.grid.hr, k=100 if m.method_name != 'ssfm' else 10)
    plot_max_values(axs[1], M.res, M.grid.N, k=100 if m.method_name != 'ssfm' else 10)
    plt.tight_layout()
    savefig(f'norm_max_{m.method_name}')
    
    _, _, Es = M.calculate_util_values()
    E, Ek, Ep = Es
    
    fig, axs = plt.subplots(1, 2, figsize=set_size(subplots=(1, 2), fraction=1, ratio='4:3'))
    axs = [thiner_border(i) for i in axs]
    
    plot_Es(axs, E, Ek, Ep, norm=False, k=100 if m.method_name != 'ssfm' else 10)
    plt.tight_layout()
    savefig(f'E_{m.method_name}')

In [ ]:
for rr in R:
    print('{}: {}, {:.2e}'.format(rr.method_name, rr.Nt, 10/rr.Nt))